In [1]:
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch import optim
import cv2
import csv
import sys
import os
from PIL import Image


In [2]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()

        # define: encoder
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 16, 3, 2),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(0.01, inplace=True),
            nn.Conv2d(16, 32, 3, 2),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.01, inplace=True),
            nn.Conv2d(32, 64, 3, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.01, inplace=True)
        )
 
        self.classifier = nn.Sequential(
            nn.Linear(320, 50),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(50, 10),
        )




    
    def forward(self, x):
        features = self.feature_extractor(x)
        features = features.view(x.shape[0], -1)
        logits = self.classifier(features)
        return logits

In [3]:
# detect is gpu available.
use_gpu = torch.cuda.is_available()
if use_gpu:
    device =torch.device('cuda:0')
else:
    device =torch.device("cpu")


# load data and normalize to [-1, 1]
trainX = np.load('./trainX.npy')
print(trainX.shape)
trainX = np.transpose(trainX, (0, 3, 1, 2))/ 255.
trainX = torch.Tensor(trainX)


# if use_gpu, send model / data to GPU.
if use_gpu:
    autoencoder.cuda()
    trainX = trainX.cuda()

# Dataloader: train shuffle = True
train_dataloader = DataLoader(trainX, batch_size=32, shuffle=True)
test_dataloader = DataLoader(trainX, batch_size=32, shuffle=False)

(9000, 32, 32, 3)


In [4]:
def loss_fn(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, size_average=False)
    # BCE = F.mse_loss(recon_x, x, size_average=False)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD, BCE, KLD






def loss_function(recon_x, x, mu, logvar):
#     BCE = F.binary_cross_entropy(recon_x, x, size_average=False)
    
    loss = nn.L1Loss(reduction='sum')
#     MSE = F.mse_loss(recon_x, x, size_average=False)
    l1_loss = loss(recon_x, x)
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return l1_loss + KLD, KLD, l1_loss

In [5]:
model = VAE()

In [9]:
# We set criterion : L1 loss (or Mean Absolute Error, MAE)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)


# Now, we train 20 epochs.
for epoch in range(1500):
    model.train()
    total_loss, best_loss = 0, 100
    """csie ta code
    for x in train_dataloader:

        latent, reconstruct = model(x)
        loss = criterion(reconstruct, x)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        cumulate_loss = loss.item() * x.shape[0]

    print(f'Epoch { "%03d" % (epoch+1) }: Loss : { "%.8f" % (cumulate_loss / trainX.shape[0])}')
    """

    for idx, image in enumerate(train_dataloader):
        reconsturct , mean, var = model(image)
        #print(reconsturct.shape,mean.shape,var.shape)
        loss, bce, kld = loss_function(reconsturct, image, mean, var)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += (loss.item() / len(train_dataloader))
        print('[Epoch %d | %d/%d] loss: %.8f' %((epoch+1), idx*32, len(train_dataloader)*32, loss.item()), end='\r')
    print("\n  Training  | Loss:%.4f \n\n" % total_loss)

[Epoch 1 | 8992/9024] loss: 851.515502933
  Training  | Loss:3498.0372 


[Epoch 2 | 8992/9024] loss: 1525.10913086
  Training  | Loss:3524.9271 


[Epoch 3 | 8992/9024] loss: 984.275817878
  Training  | Loss:3452.3011 


[Epoch 4 | 8992/9024] loss: 1303.05590820
  Training  | Loss:3501.6713 


[Epoch 5 | 8992/9024] loss: 878.359069829
  Training  | Loss:3490.7313 


[Epoch 6 | 8992/9024] loss: 1051.80712891
  Training  | Loss:3472.7910 


[Epoch 7 | 8992/9024] loss: 996.740295415
  Training  | Loss:3447.7288 


[Epoch 8 | 8992/9024] loss: 830.584167480
  Training  | Loss:3490.6267 


[Epoch 9 | 8992/9024] loss: 1004.85620117
  Training  | Loss:3463.8416 


[Epoch 10 | 8992/9024] loss: 1095.71691895
  Training  | Loss:3522.2915 


[Epoch 11 | 8992/9024] loss: 1598.37158203
  Training  | Loss:3483.8668 


[Epoch 12 | 8992/9024] loss: 799.922607422
  Training  | Loss:3474.4562 


[Epoch 13 | 8992/9024] loss: 1011.86444092
  Training  | Loss:3491.9109 


[Epoch 14 | 8992/9024] loss: 1160.

[Epoch 218 | 8992/9024] loss: 1062.74645996
  Training  | Loss:3414.5719 


[Epoch 219 | 8992/9024] loss: 891.661315920
  Training  | Loss:3408.7714 


[Epoch 220 | 8992/9024] loss: 1249.51025391
  Training  | Loss:3397.8026 


[Epoch 221 | 8992/9024] loss: 1052.55920410
  Training  | Loss:3384.0579 


[Epoch 222 | 8992/9024] loss: 1172.29772949
  Training  | Loss:3370.3172 


[Epoch 223 | 8992/9024] loss: 981.078186048
  Training  | Loss:3387.9638 


[Epoch 224 | 8992/9024] loss: 1259.07177734
  Training  | Loss:3405.3046 


[Epoch 225 | 8992/9024] loss: 1118.61486816
  Training  | Loss:3381.8136 


[Epoch 226 | 8992/9024] loss: 1513.37988281
  Training  | Loss:3381.6001 


[Epoch 227 | 8992/9024] loss: 1171.89416504
  Training  | Loss:3396.4887 


[Epoch 228 | 8992/9024] loss: 1263.78869629
  Training  | Loss:3376.6015 


[Epoch 229 | 8992/9024] loss: 1030.07690430
  Training  | Loss:3379.1813 


[Epoch 230 | 8992/9024] loss: 783.670654303
  Training  | Loss:3358.0424 


[Epoch 231 |

[Epoch 326 | 8992/9024] loss: 1043.45458984
  Training  | Loss:3338.1970 


[Epoch 327 | 8992/9024] loss: 976.125732422
  Training  | Loss:3342.9570 


[Epoch 328 | 8992/9024] loss: 809.192810062
  Training  | Loss:3351.4441 


[Epoch 329 | 8992/9024] loss: 963.396179203
  Training  | Loss:3362.0759 


[Epoch 330 | 8992/9024] loss: 1364.94067383
  Training  | Loss:3351.3783 


[Epoch 331 | 8992/9024] loss: 956.124511724
  Training  | Loss:3382.6487 


[Epoch 332 | 8992/9024] loss: 1171.33557129
  Training  | Loss:3308.1791 


[Epoch 333 | 8992/9024] loss: 878.784790049
  Training  | Loss:3367.8387 


[Epoch 334 | 8992/9024] loss: 1034.35498047
  Training  | Loss:3341.6071 


[Epoch 335 | 8992/9024] loss: 1256.91735840
  Training  | Loss:3331.3833 


[Epoch 336 | 8992/9024] loss: 960.797363285
  Training  | Loss:3408.2332 


[Epoch 337 | 8992/9024] loss: 844.273986825
  Training  | Loss:3367.1678 


[Epoch 338 | 8992/9024] loss: 930.779602050
  Training  | Loss:3347.2371 


[Epoch 339 |

[Epoch 434 | 8992/9024] loss: 997.762146008
  Training  | Loss:3278.8163 


[Epoch 435 | 8992/9024] loss: 1067.60717773
  Training  | Loss:3334.2494 


[Epoch 436 | 8992/9024] loss: 756.799377446
  Training  | Loss:3356.2752 


[Epoch 437 | 8992/9024] loss: 1025.75610352
  Training  | Loss:3307.2937 


[Epoch 438 | 8992/9024] loss: 894.236145020
  Training  | Loss:3321.5772 


[Epoch 439 | 8992/9024] loss: 932.516662601
  Training  | Loss:3347.2905 


[Epoch 440 | 8992/9024] loss: 981.675537117
  Training  | Loss:3321.0885 


[Epoch 441 | 8992/9024] loss: 947.317260741
  Training  | Loss:3373.1619 


[Epoch 442 | 8992/9024] loss: 997.264038091
  Training  | Loss:3318.1030 


[Epoch 443 | 8992/9024] loss: 1288.18139648
  Training  | Loss:3282.9663 


[Epoch 444 | 8992/9024] loss: 1187.31335449
  Training  | Loss:3367.9972 


[Epoch 445 | 8992/9024] loss: 1234.10314941
  Training  | Loss:3315.1944 


[Epoch 446 | 8992/9024] loss: 1054.83142090
  Training  | Loss:3336.2910 


[Epoch 447 |

[Epoch 542 | 8992/9024] loss: 1144.27868652
  Training  | Loss:3319.8751 


[Epoch 543 | 8992/9024] loss: 1345.22619629
  Training  | Loss:3325.8989 


[Epoch 544 | 8992/9024] loss: 873.858642582
  Training  | Loss:3293.4011 


[Epoch 545 | 8992/9024] loss: 1115.42919922
  Training  | Loss:3308.8303 


[Epoch 546 | 8992/9024] loss: 1046.65185547
  Training  | Loss:3283.3746 


[Epoch 547 | 8992/9024] loss: 867.463806150
  Training  | Loss:3286.0862 


[Epoch 548 | 8992/9024] loss: 1174.99951172
  Training  | Loss:3299.5344 


[Epoch 549 | 8992/9024] loss: 787.450988773
  Training  | Loss:3316.7732 


[Epoch 550 | 8992/9024] loss: 838.168823241
  Training  | Loss:3319.6071 


[Epoch 551 | 8992/9024] loss: 967.579589849
  Training  | Loss:3291.8284 


[Epoch 552 | 8992/9024] loss: 836.193420412
  Training  | Loss:3293.4102 


[Epoch 553 | 8992/9024] loss: 882.919372563
  Training  | Loss:3335.9207 


[Epoch 554 | 8992/9024] loss: 1333.08862305
  Training  | Loss:3299.2635 


[Epoch 555 |

KeyboardInterrupt: 

In [10]:
# Collect the latents and stdardize it.
latents = []
latent_sapce = []
for x in test_dataloader:
    _,mu,var = model(x)
    mu = mu.detach().cpu().numpy()
    for i in range(mu.shape[0]):
        latent_sapce.append(mu[i])
        
print('latent_space finish')
latent_space = np.asarray(latent_sapce)

print(latent_space.shape)
latents = (latent_space - np.mean(latent_space, axis=0)) / np.std(latent_space, axis=0)

# Use PCA to lower dim of latents and use K-means to clustering.
pca = PCA(n_components=32, copy=False, whiten=True, svd_solver='full')
latent_vec = pca.fit_transform(latents)
latent_vec = TSNE(n_components = 3,verbose=5).fit_transform(latent_vec)
#result = SpectralClustering(n_clusters=2,random_state = 2, n_init=10, gamma=1.0).fit_predict(latent_vec)
result = KMeans(n_clusters=2, random_state=2, max_iter=1000).fit(latent_vec).labels_


"""
latents = PCA(n_components=16).fit_transform(latents)
result = KMeans(n_clusters = 2).fit(latents).labels_
"""
# We know first 5 labels are zeros, it's a mechanism to check are your answers
# need to be flipped or not.
print(np.sum(result[:5]))
if np.sum(result[:5]) >= 3:
    result = 1 - result
""""
if np.sum(result[:5]) != 0 or np.sum(result[:5])!=5:
    print("redo")
"""
# Generate your submission
df = pd.DataFrame({'id': np.arange(0,len(result)), 'label': result})
df.to_csv('gogogo_11_19_14_30.csv',index=False)

latent_space finish
(9000, 512)
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 9000 samples in 0.007s...
[t-SNE] Computed neighbors for 9000 samples in 4.781s...
[t-SNE] Computed conditional probabilities for sample 1000 / 9000
[t-SNE] Computed conditional probabilities for sample 2000 / 9000
[t-SNE] Computed conditional probabilities for sample 3000 / 9000
[t-SNE] Computed conditional probabilities for sample 4000 / 9000
[t-SNE] Computed conditional probabilities for sample 5000 / 9000
[t-SNE] Computed conditional probabilities for sample 6000 / 9000
[t-SNE] Computed conditional probabilities for sample 7000 / 9000
[t-SNE] Computed conditional probabilities for sample 8000 / 9000
[t-SNE] Computed conditional probabilities for sample 9000 / 9000
[t-SNE] Mean sigma: 1.285854
[t-SNE] Computed conditional probabilities in 0.215s
[t-SNE] Iteration 50: error = 91.6750641, gradient norm = 0.0000000 (50 iterations in 19.522s)
[t-SNE] Iteration 50: gradient norm 0.000000. Finished.


In [11]:
torch.save(model,"vae_11181612")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type VAE. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
